ディレクトリ・モデルの指定

In [ ]:
# リポジトリを置くところ
home_dir = '/content/drive/MyDrive'
%cd {home_dir}/Yamashita-yolov7/
!pip install -r requirements.txt

# データを入れたディレクトリの名前
data_dir='pumpkin_640_480'

# ベースモデル: yolov7-tiny yolov7 yolov7x yolov7-w6 yolov7-e6 yolov7-d6 yolov7-e6e
model_type = "yolov7-tiny"
# 必要に応じてインストール
# !wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/{model_type}.pt

YAMLの設定ファイルを生成する

In [14]:
lines = [
  "train: workspace/" + data_dir + "/dataset/images/train",
  "val: workspace/" + data_dir + "/dataset/images/val",
  "test:  # optional",
  "nc: 1  # number of classes",
  "names: ['pumpkin']",
]
with open('custom_cfg.yaml', mode='w') as f:
  for line in lines:
    f.write(line + '\n')

学習

In [ ]:
# メモリ不足で落ちる場合はworkersを下げ，それでもダメならbatch_sizeを下げる
img_size = 640
batch_size = 32
workers = 8
epochs = 100
hyper_param = 'data/hyp.scratch.tiny.yaml'

!python train.py \
  --workers {workers} \
  --device 0 \
  --batch-size {batch_size} \
  --epochs {epochs} \
  --data custom_cfg.yaml \
  --img {img_size} \
  --cfg cfg/training/{model_type}.yaml \
  --weights {model_type}.pt \
  --name {model_type} \
  --hyp {hyper_param}

ONNXフォーマットに書き出し

In [ ]:
!pip install onnx 
!pip install onnxruntime
!pip install onnx-simplifier

!python export.py \
  --weights runs/train/yolov7-tiny/weights/best.pt \
  --grid --end2end --simplify \
  --topk-all 100 \
  --iou-thres 0.1 \
  --conf-thres 0.1 \
  --img-size 640 \
  --max-wh 640